In [ ]:
%matplotlib inline
import os
import shutil
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import pyemu

In [ ]:
sc = pyemu.Schur("freyberg_zones.jcb")

In [ ]:
sc.get_parameter_summary()

In [ ]:
sc.get_forecast_summary()

# Parameter by parameter, fix the parameter and see the impact on forecast uncertainty (simulating perfect knowledge of the parameter)

In [ ]:
sc_par_contribution = sc.get_par_contribution()
sc_par_contribution.drop('percent_reduce', axis=1, level=1)

# We can see how much worse forecasts get without an observation (flipping that around, how valuable is each observation to the forecasts)

In [ ]:
sc_removed_import = sc.get_removed_obs_importance()
sc_removed_import_scaled = sc_removed_import.copy()
sc_removed_import

In [ ]:
# let's scale this
for cc in sc_removed_import.columns:
    sc_removed_import_scaled[cc]  =  100*(sc_removed_import_scaled[cc]-sc_removed_import_scaled[cc].loc['base'])/ \
        sc_removed_import_scaled[cc].loc['base']

In [ ]:
for cc in sc_removed_import_scaled.columns:
    plt.figure()
    sc_removed_import_scaled[cc].plot(kind='bar')
    plt.title(cc)

In [ ]:
sc = pyemu.Schur(jco="freyberg_zones_burn_potobs.jcb")
sc.get_forecast_summary()

In [ ]:
obs = sc.pst.observation_data
pobs_names = list(obs.loc[obs.obgnme=="potobs","obsnme"])

In [ ]:
df = sc.get_added_obs_importance(obslist_dict={oname:oname for oname in pobs_names},
                                base_obslist=sc.pst.nnz_obs_names,reset_zero_weight=True)

In [ ]:
df.min()

In [ ]:
df.max()

In [ ]:
df1 = sc.get_added_obs_importance(obslist_dict={oname:oname for oname in pobs_names},reset_zero_weight=True)

In [ ]:
df1.min()

In [ ]:
df1.max()

In [ ]:
def plot_added_worth(df):
    df_base = df.loc["base",:]
    df_frac = 100.0 * (1.0 - (df / df_base))
    df_frac = df_frac.loc[pobs_names,:]
    df_frac.loc[:,"row"] = df_frac.index.map(lambda x: int(x[2:4]))
    df_frac.loc[:,"column"] = df_frac.index.map(lambda x: int(x[-2:]))

    for forecast in df_frac.columns:
        if forecast in ["row","column"]:
            continue
        arr = np.zeros((df_frac.row.max(),df_frac.column.max())) - 1.0
        for r,c,f in zip(df_frac.row,df_frac.column,df_frac.loc[:,forecast]):
            arr[r-1,c-1] = f
        arr = np.ma.masked_where(arr < 0,arr)
        fig = plt.figure(figsize=(6,10))
        ax = plt.subplot(111)
        c = ax.imshow(arr,interpolation="nearest",alpha=0.5)
        #ax.imshow(zn_arr,interpolation="nearest",alpha=0.5)
        plt.colorbar(c)
        ax.set_title(forecast)
        plt.show()

    

In [ ]:
zn_arr = np.loadtxt("hk.zones")
fig = plt.figure(figsize=(6,10))
ax = plt.subplot(111)
c = ax.imshow(zn_arr,interpolation="nearest",alpha=0.5)
plt.show()

In [ ]:
plot_added_worth(df)

In [ ]:
plot_added_worth(df1)

In [ ]:
pst = pyemu.Pst("freyberg_zones_potobs.pst")
pst.template_files[1] = "hk.ref.tpl.alt"
pst.template_files
pst.control_data.noptmax = -1
pst.write("freyberg_zones_alt.pst")
os.system("pestpp freyberg_zones_alt.pst")

In [ ]:
sc = pyemu.Schur(jco="freyberg_zones_alt.jcb")
df = sc.get_added_obs_importance(obslist_dict={oname:oname for oname in pobs_names},reset_zero_weight=True)

In [ ]:
zn_arr = np.loadtxt("kzone.ref.alt")
fig = plt.figure(figsize=(6,10))
ax = plt.subplot(111)
c = ax.imshow(zn_arr,interpolation="nearest",alpha=0.5)
plt.show()

In [ ]:
plot_added_worth(df)